# 0) Setup

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask

plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 120

print('xarray', xr.__version__)


xarray 2025.9.0


In [2]:
from dask.distributed import Client
client = Client()  # set up local cluster on my machine
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [3]:
import gcsfs
fs = gcsfs.GCSFileSystem(token="anon")
# Should list arrays/metadata keys under the store
print(fs.ls("gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3")[:10])

['gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/.zattrs', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/.zgroup', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/.zmetadata', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/100m_u_component_of_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/100m_v_component_of_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/10m_u_component_of_neutral_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/10m_u_component_of_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/10m_v_component_of_neutral_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/10m_v_component_of_wind', 'gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3/10m_wind_gust_since_previous_post_processing']


# 1) Subsetting Precipitation data of Charleston, SC for 20 years

In [4]:
#Opening data
ds = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks=None,
    storage_options=dict(token='anon'),
)
ds = ds.sel(time=slice(ds.attrs['valid_time_start'], ds.attrs['valid_time_stop']))

#Define var as precipitation to subset data later.
var = 'total_precipitation'
ds

<xarray.Dataset> Size: 2PB
Dimensions:                                                          (
                                                                      time: 748008,
                                                                      latitude: 721,
                                                                      longitude: 1440,
                                                                      level: 37)
Coordinates:
  * latitude                                                         (latitude) float32 3kB ...
  * level                                                            (level) int64 296B ...
  * longitude                                                        (longitude) float32 6kB ...
  * time                                                             (time) datetime64[ns] 6MB ...
Data variables: (12/273)
    100m_u_component_of_wind                                         (time, latitude, longitude) float32 3TB ...
    100m_v_component_of_wind                                         (time, latitude, longitude) float32 3TB ...
    10m_u_component_of_neutral_wind                                  (time, latitude, longitude) float32 3TB ...
    10m_u_component_of_wind                                          (time, latitude, longitude) float32 3TB ...
    10m_v_component_of_neutral_wind                                  (time, latitude, longitude) float32 3TB ...
    10m_v_component_of_wind                                          (time, latitude, longitude) float32 3TB ...
    ...                                                               ...
    wave_spectral_directional_width_for_swell                        (time, latitude, longitude) float32 3TB ...
    wave_spectral_directional_width_for_wind_waves                   (time, latitude, longitude) float32 3TB ...
    wave_spectral_kurtosis                                           (time, latitude, longitude) float32 3TB ...
    wave_spectral_peakedness                                         (time, latitude, longitude) float32 3TB ...
    wave_spectral_skewness                                           (time, latitude, longitude) float32 3TB ...
    zero_degree_level                                                (time, latitude, longitude) float32 3TB ...
Attributes:
    last_updated:           2025-09-22 01:57:27.339380+00:00
    valid_time_start:       1940-01-01
    valid_time_stop:        2025-04-30
    valid_time_stop_era5t:  2025-09-16

In [5]:
#Subsetting data to include precipitation during the first 20 years of my life :)
dss = ds[var]

dss = dss.sel(time=slice('2000-08-25 00:00', '2020-08-24 23:59'))

dss

<xarray.DataArray 'total_precipitation' (time: 175320, latitude: 721,
                                         longitude: 1440)> Size: 728GB
[182024236800 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 1MB 2000-08-25 ... 2020-08-24T23:00:00
Attributes:
    long_name:   Total precipitation
    short_name:  tp
    units:       m

In [6]:
#Subset the data further to include a 5x5 grid of Charleston, SC
lons = [280, 285]
lats = [30, 35]

dsss = dss.sel(latitude=slice(lats[1],lats[0]), longitude=slice(lons[0],lons[1]))

dsss

<xarray.DataArray 'total_precipitation' (time: 175320, latitude: 21,
                                         longitude: 21)> Size: 309MB
[77316120 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float32 84B 35.0 34.75 34.5 34.25 ... 30.5 30.25 30.0
  * longitude  (longitude) float32 84B 280.0 280.2 280.5 ... 284.5 284.8 285.0
  * time       (time) datetime64[ns] 1MB 2000-08-25 ... 2020-08-24T23:00:00
Attributes:
    long_name:   Total precipitation
    short_name:  tp
    units:       m

In [ ]:
#Convert hourly precipitation data to daily data

dsss_daily = dsss.resample(time='1D').sum() *1000

dsss_daily

In [ ]:
dsss_daily.sel(latitude=32, longitude=-80+360).plot

In [ ]:
dsss_daily.to_netcdf('era5_CHS_precip_08252000_08242020.nc')